# Object Creation: Self-Report Stress Objects from Mood Events

This notebook demonstrates how to create stress self-report objects from mood events in the OCED data. Each stress self-report object represents a stress value reported in a mood event, and is linked to:
<ul>
<li>The mood event that reported the stress
<li>The day on which it occurred
<li>The user who reported it
<li>The nearest notification within a specified time period (if any exists)

## 1. Setup

In [1]:

import sys
sys.path.append('..')
from pathlib import Path
import pandas as pd
from datetime import timedelta
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.stress_objects import StressObjectManager

## 2. Load OCED Data

In [2]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"player_107631_oced_data_time_bouts_notifications.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

# Diagnostic code to inspect notification events
mood_events = [
    notif for notif in data_dict.get('behaviorEvents', [])
    if notif['behaviorEventType'] == 'mood'
]
print(len(mood_events))
mood_events

Number of behavior events: 498
Number of objects: 273
76


[{'id': '67b8d80e-fb50-4037-9c3a-8db93746fc89',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T13:00:00.886000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 6},
   {'name': 'stress', 'value': 4}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_type': 'behavior'},
 {'id': '02a64aa1-1d70-4d0d-bb40-c94a0e515f19',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T09:55:34.238000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 3},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 7}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'}],
  'event_

## 3. Initialize Stress Object Manager
Create an instance of the StressObjectManager class and add the stress self-report object type to the data.

In [3]:
# Initialize the stress object manager
manager = StressObjectManager()

# Add stress self-report object type to the data
data = manager.create_stress_object_type(data_dict)

# Print object types to verify
print("Object types in data:")
for obj_type in data.get('objectTypes', []):
    print(f"- {obj_type['name']}")

Stress self-report object type added.
Object types in data:
- player
- intervention
- day
- week
- physical_activity_bout
- notification
- stress_self_report


## 4. Create Stress Self-Report Objects
Now we'll create stress self-report objects from the mood events. We'll use a 1-hour time window to look for nearby notifications.

In [4]:
# Get the user ID from the data
user_objects = [obj for obj in data.get('objects', []) if obj['type'] == 'player']
if not user_objects:
    raise ValueError("No user object found in the data")
user_id = user_objects[0]['id']
print(f"Using user ID: {user_id}")

# Create stress self-report objects with a 1-hour time window
time_period = timedelta(hours=5)
extended_data, stress_objects = manager.create_stress_objects(
    data=data,
    user_id=user_id,
    time_period=time_period
)

Using user ID: d360ac20-326e-4f6f-8113-baa47ef536ef

Found 76 mood events
Found 0 existing stress self-report objects


Processing mood events:   0%|          | 0/76 [00:00<?, ?it/s]

Creating new stress self-report object 39c34d2c-02ce-4720-8b38-e9a70009fdf8 for mood event
Creating new stress self-report object 75422593-efdc-44e3-a93c-0ea025643582 for mood event


Processing mood events:   4%|▍         | 3/76 [00:00<00:03, 22.83it/s]

Creating new stress self-report object 627f800e-8752-477c-bc91-972d74532435 for mood event
Creating new stress self-report object 1c9fd423-07f8-4d44-9387-86db57f7e234 for mood event
Creating new stress self-report object e11fee9a-3114-43b4-8bb0-2e7e4a25ae7f for mood event


...

Linking stress self-report object to nearby notification 4da9b04b-af72-4f3b-8ba1-a2c8e7ab5300
Creating new stress self-report object 0dd0cdb2-6b7a-4e45-acb8-a8ce22dc1a16 for mood event
Linking stress self-report object to nearby notification ce91efe9-0d63-4c1a-94aa-f3d5db4ac3ce
Creating new stress self-report object ab5909bf-565f-4c92-9934-090aaa6284cd for mood event
Linking stress self-report object to nearby notification 0e48529f-381b-4516-9d3b-2329dddb0774
Creating new stress self-report object 4bd07c00-a3e7-45fa-8e32-b4da3fe406b1 for mood event
Linking stress self-report object to nearby notification 5196c02c-5149-466f-bc2b-b5f240965c49
Creating new stress self-report object 591ef5f5-ec5b-4ee2-9aab-d30fb5ca9ddc for mood event
Linking stress self-report object to nearby notification 9fde7b23-2295-4715-bbb7-2048a0f9a685
Creating new stress self-report object 7c0a6610-7818-4dcb-bd08-57f1e8af1631 for mood event
Linking stress self-report object to nearby notification 9fde7b23-2295-4715

Processing mood events:  91%|█████████ | 69/76 [00:02<00:00, 24.89it/s]

Linking stress self-report object to nearby notification 2ee0638f-d410-4550-ae53-30766494e288
Creating new stress self-report object 8a355819-29e9-455d-be9f-2720bd3bb698 for mood event
Linking stress self-report object to nearby notification 0a0c2c75-d65f-43d8-9a94-b5726ba07ddf
Creating new stress self-report object f6444a80-eb9d-4074-bdb9-5bdb49397c5f for mood event
Linking stress self-report object to nearby notification c12e308b-03b3-4865-b62d-15ff5de308fc
Creating new stress self-report object 5380be9d-ab08-4b7e-bfdf-0ea1ac6bb60c for mood event
Linking stress self-report object to nearby notification f5acbce4-4c5c-4cbb-9aa8-b6981cb8bdf7
Creating new stress self-report object 3d821356-663d-444b-b3ae-5591cd91301c for mood event
Linking stress self-report object to nearby notification 1650094d-b856-496f-acc7-bbbb7d166b96
Creating new stress self-report object a6018faf-450e-488f-8316-a6991aa7eea5 for mood event
Linking stress self-report object to nearby notification 1a5e2fe5-2588-4f2f

Processing mood events: 100%|██████████| 76/76 [00:03<00:00, 24.24it/s]

Linking stress self-report object to nearby notification 6295e2a8-5fe3-41dd-89e9-aa34a8c1a543
Creating new stress self-report object 646c059a-a890-41f1-900f-27f6bb3febb8 for mood event
Linking stress self-report object to nearby notification c561b72f-bc07-4530-ac1e-f27727992752
Creating new stress self-report object 4f6de4cd-252c-4908-a98c-b69c0835a2be for mood event
Linking stress self-report object to nearby notification 1470133f-b15f-4651-b617-82b20a7612eb
Creating new stress self-report object 630def80-0ca5-4169-abbd-e04808cd9ee5 for mood event
Linking stress self-report object to nearby notification 096570e3-7618-4ed0-9a4e-89ae19c02061
Creating new stress self-report object 67b66e50-dace-44db-8bb6-dd3b01bf4b3d for mood event
Linking stress self-report object to nearby notification 89e0c126-dcd9-429d-b468-08c239ac06cc
Creating new stress self-report object b007b627-8e70-4a80-b511-be98b3e2bf3a for mood event
Linking stress self-report object to nearby notification a61d3d86-ba05-4fe9

## 5. Analyze Created Objects
Let's analyze the created stress self-report objects and their relationships.

In [5]:
# Count stress self-report objects with notification relationships
stress_with_notifications = [
    obj for obj in stress_objects
    if any(rel['qualifier'] == 'follows_notification' for rel in obj.get('relationships', []))
]

print(f"Stress self-report objects analysis:")
print(f"- Total stress self-report objects: {len(stress_objects)}")
print(f"- Stress self-report objects with notification relationships: {len(stress_with_notifications)}")
print(f"- Percentage with notifications: {len(stress_with_notifications)/len(stress_objects)*100:.1f}%")

# Look at a sample stress self-report object
if stress_objects:
    sample = stress_objects[0]
    print("\nSample stress self-report object:")
    print(f"ID: {sample['id']}")
    print("Attributes:")
    for attr in sample['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in sample.get('relationships', []):
        print(f"- {rel['qualifier']} -> {rel['id']}")

Stress self-report objects analysis:
- Total stress self-report objects: 76
- Stress self-report objects with notification relationships: 52
- Percentage with notifications: 68.4%

Sample stress self-report object:
ID: 39c34d2c-02ce-4720-8b38-e9a70009fdf8
Attributes:
- stress_value: 6.0 (at 2025-03-20T15:39:28.698000)

Relationships:
- occurred_on -> f7397705-b95e-4127-9090-32e0297be74a
- reported_by -> d360ac20-326e-4f6f-8113-baa47ef536ef


## 6. Save Extended Data
Finally, let's save the extended data with the new stress self-report objects.

In [6]:
# Save the extended data
manager.save_extended_data("player_107631_oced_data_time_bouts_notifications_stress.json", extended_data)

print("\nData saved successfully!")


Saving extended data with:
- 76 stress self-report objects
- 76 mood events
- 349 total objects
- 498 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\player_107631_oced_data_time_bouts_notifications_stress.json

Data saved successfully!


In [7]:
# Count stress self-report objects with notification relationships
data_objects = [
    obj for obj in extended_data.get('objects', [])
    if obj['type'] == 'stress_self_report'
]

print(len(data_objects))
data_objects

76


[{'id': '39c34d2c-02ce-4720-8b38-e9a70009fdf8',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 6.0,
    'time': '2025-03-20T15:39:28.698000'}],
  'relationships': [{'id': 'f7397705-b95e-4127-9090-32e0297be74a',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': '75422593-efdc-44e3-a93c-0ea025643582',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-21T11:12:32.086000'}],
  'relationships': [{'id': '109925f4-92f9-4171-8de8-c158ce3c4bb9',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'reported_by'}]},
 {'id': '627f800e-8752-477c-bc91-972d74532435',
  'type': 'stress_self_report',
  'attributes': [{'name': 'stress_value',
    'value': 5.0,
    'time': '2025-03-24T12:16:29.4

In [8]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('objectTypes', [])
]

print(len(data))
data

7


[{'name': 'player', 'attributes': [{'name': 'id', 'type': 'string'}]},
 {'name': 'intervention',
  'attributes': [{'name': 'goal', 'type': 'string'},
   {'name': 'start_date', 'type': 'string'},
   {'name': 'end_date', 'type': 'string'}]},
 {'name': 'day',
  'attributes': [{'name': 'date', 'type': 'string'},
   {'name': 'day_of_week', 'type': 'string'}]},
 {'name': 'week',
  'attributes': [{'name': 'week_start_date', 'type': 'string'},
   {'name': 'week_number', 'type': 'integer'},
   {'name': 'year', 'type': 'integer'}]},
 {'name': 'physical_activity_bout',
  'attributes': [{'name': 'bout_type', 'type': 'string'}]},
 {'name': 'notification',
  'attributes': [{'name': 'last_action', 'type': 'string'}]},
 {'name': 'stress_self_report',
  'attributes': [{'name': 'stress_value', 'type': 'number'}]}]

In [9]:
# Count stress self-report objects with notification relationships
data = [
    obj for obj in extended_data.get('behaviorEvents', [])
    if obj['behaviorEventType'] == 'mood'
]

print(len(data))
data

76


[{'id': '67b8d80e-fb50-4037-9c3a-8db93746fc89',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T13:00:00.886000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 6},
   {'name': 'arousal', 'value': 6},
   {'name': 'stress', 'value': 4}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id': '8d47453a-6a5c-454d-806a-971b71a11d8f',
    'type': 'object',
    'qualifier': 'occurred_on'},
   {'id': 'b007b627-8e70-4a80-b511-be98b3e2bf3a',
    'type': 'object',
    'qualifier': 'reports_stress'}],
  'event_type': 'behavior'},
 {'id': '02a64aa1-1d70-4d0d-bb40-c94a0e515f19',
  'behaviorEventType': 'mood',
  'time': '2025-05-25T09:55:34.238000',
  'behaviorEventTypeAttributes': [{'name': 'valence', 'value': 3},
   {'name': 'arousal', 'value': 4},
   {'name': 'stress', 'value': 7}],
  'relationships': [{'id': 'd360ac20-326e-4f6f-8113-baa47ef536ef',
    'type': 'object',
    'qualifier': 'source'},
   {'id